# Dette er markdown.
## Her kan jeg sette in SUBTITLE


In [1]:
this_is_python = "I 💗 python"
print(this_is_python)

I 💗 python


In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow import keras

# Onehot encoding of values

In [3]:
x_data = " hello world"
y_data = "hello world "
int_char = dict((i, c) for (i,c) in enumerate("".join(set(x_data))))
dictonary = dict((c, i) for (i,c) in enumerate("".join(set(x_data))))

In [4]:
x_train = [list(keras.utils.to_categorical(list(dictonary[val] for val in x_data)))]
y_train = [list(keras.utils.to_categorical(list(dictonary[val] for val in y_data)))]

In [5]:
print(np.array(x_train).shape)
print(x_train[0][0])

(1, 12, 8)
[0. 0. 1. 0. 0. 0. 0. 0.]


# placeholders

In [6]:
units_in_LSTM = 128
batch_size = 8
epochs = 1000

x = tf.placeholder(shape = [None, None, batch_size], dtype = tf.float32, name = "x")
y_model = tf.placeholder(shape = [None, None, batch_size], dtype = tf.float32, name = "y")
state_value = tf.placeholder(dtype = tf.int32, shape = [])

w = tf.Variable(tf.random_uniform([units_in_LSTM, batch_size]),shape=[units_in_LSTM,batch_size], dtype = tf.float32, name = "weight")
b = tf.Variable(tf.random_uniform([batch_size]), dtype = tf.float32, shape=[batch_size], name = "bias")

In [7]:
cell = tf.contrib.rnn.BasicLSTMCell(units_in_LSTM)
in_state = cell.zero_state(state_value, dtype = tf.float32)

W0917 13:12:00.439109 4512822720 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

W0917 13:12:00.440053 4512822720 deprecation.py:323] From <ipython-input-7-ff1732769819>:1: BasicLSTMCell.__init__ (from tensorflow.python.ops.rnn_cell_impl) is deprecated and will be removed in a future version.
Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.


In [8]:
lstm, out_state = tf.nn.dynamic_rnn(cell, x, initial_state=in_state)

logits = tf.nn.bias_add(tf.einsum('bts,se->bte', lstm, w), b) 
softie = tf.nn.softmax(logits)

loss = tf.losses.softmax_cross_entropy(softie, logits)
optimizer = tf.train.RMSPropOptimizer(learning_rate = 0.09)
trainer = optimizer.minimize(loss)

init = tf.global_variables_initializer()

W0917 13:12:00.467050 4512822720 deprecation.py:323] From <ipython-input-8-a5f8a652e51d>:1: dynamic_rnn (from tensorflow.python.ops.rnn) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
W0917 13:12:00.519475 4512822720 deprecation.py:506] From /Users/williamjacobsen/anaconda3/envs/tensorflow/lib/python3.6/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0917 13:12:00.526454 4512822720 deprecation.py:506] From /Users/williamjacobsen/anaconda3/envs/tensorflow/lib/python3.6/site-packages/tensorflow/python/ops/rnn_cell_impl.py:738: calling Zeros.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be remov

In [9]:
with tf.Session() as sess:
    sess.run(init)
    zero_state = sess.run(in_state, {state_value: 1})

    for i in range(500):
        feed = {state_value: 1, x: x_train, y_model: y_train, in_state : zero_state}
        sess.run(trainer, feed)
        
        if i % 10 == 0:
            print("epoch", i)
            print("loss", sess.run(loss, {state_value: 1, x: x_train, y_model: y_train, in_state: zero_state}))
            # Generate characters from the initial characters ' h'
            state = sess.run(in_state, {state_value: 1})
            text = ' h'
            y, state = sess.run([softie, out_state], {state_value: 1, x: [[x_train[0][0]]], in_state: state})  # ' '
            y, state = sess.run([softie, out_state], {state_value: 1, x: [[x_train[0][1]]], in_state: state})  # 'h'
        
            text += int_char[y.argmax()]
            for c in range(50):
                y, state = sess.run([softie, out_state], {state_value: 1, x: [[x_train[0][y.argmax()]]], in_state: state})
                text += int_char[y[0].argmax()]
            print(text)

epoch 0
loss 2.052191
 hhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhh
epoch 10
loss 2.052191
 hhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhh
epoch 20
loss 2.052191
 hhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhh
epoch 30
loss 2.052191
 hhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhh
epoch 40
loss 2.052191
 hhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhh
epoch 50
loss 2.052191
 hhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhh
epoch 60
loss 2.052191
 hhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhh
epoch 70
loss 2.052191
 hhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhh
epoch 80
loss 2.052191
 hhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhh
epoch 90
loss 2.0521908
 hhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhh
epoch 100
loss 2.0521908
 hhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhh
epoch 110
loss 2.0521908
 hhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhh
epoch 120
loss 2.052191
 hhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhh